In [2]:
import librosa.display
import librosa.util
import matplotlib.pyplot as plt
from IPython.display import Audio
import os
import scipy
import numpy as np
import utils.synthesis_util as su
import utils.model_util as mu
from utils.metrics import Metrics

## Examine dataset

In [3]:
audio_files = su.load_files()

### Clean audio

In [4]:
y_clean,sr = su.get_audio(audio_id=69,audio_files=audio_files)
Audio(y_clean,rate=sr)

## Noisy audio

In [5]:
y_noise, sr, noise = su.get_noisy_audio(audio_id=69,audio_files=audio_files)
Audio(y_noise,rate=sr)

In [6]:
win = scipy.signal.hamming(256,"periodic")
magnitude,phase = su.audio_to_sttft(y_noise,win)

In [7]:
predictors = su.get_predictors(magnitude)

In [8]:
predictors = np.array(predictors)
predictors.shape

(324, 129, 8)

## Load module

In [9]:
model = mu.load_model(n_features=129,n_segments=8,model_to_test="Baseline_FullyConnected/0505_130215")

## Denoise

In [10]:
length = len(y_noise)

In [11]:
sample = su.obtain_sample(predictors)

In [12]:
# print(model)
audio_rec = su.denoise_audio(model,sample, magnitude, phase,win,length)
Audio(audio_rec,rate=sr)

# Calculate metrics

In [13]:
magnitude_clean,phase_clean = su.audio_to_sttft(y_clean,win)
magnitude_noise,phase_noise = su.audio_to_sttft(y_noise,win)
magnitude_denoised,phase_denoised = su.audio_to_sttft(audio_rec,win)
print(f"SNR_db for clean: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_clean, sr)))}")
print(f"SNR_db for noise: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_noise, sr)))}")
print(f"SNR_db for denoised: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_denoised, sr)))}")


SNR_db for clean: 30.01601325811101
SNR_db for noise: 27.164033517997726
SNR_db for denoised: 30.67187643826303
